In [12]:
from os import walk, getcwd, path
import re
from datetime import date
import os.path, time
import pandas as pd
import glob

In [14]:
## Se define las variables y la ruta
today= date.today()
df = pd.DataFrame(columns=['Archivo','Fecha'])
raiz = '../../Downloads/Veritrade_PMARTINTO@VILLAANDINA.COM_**'
rutas = glob.glob(raiz, recursive=True)


#Se define la iteraccion en donde se guarda los archivos descargados
for ruta in rutas:
    #nombre, extension = os.path.splitext(ruta)
    tiempo2=pd.to_datetime(time.ctime(os.path.getmtime(ruta)))
    df=df.append({'Archivo':ruta,'Fecha':tiempo2},ignore_index=True)

df =   df[(df['Fecha'] >= pd.to_datetime(today))]#& (df['Fecha'] <= '2019-06-1')

#Se genera el DataframeFinal antes de subirlo
pdfinal = pd.DataFrame(columns=['Partida Aduanera','Descripcion de la Partida Aduanera','Aduana','DUA','Fecha','Cod. Tributario',
                                'Exportador','Importador','Kg Bruto','Kg Neto','Qty 1','Und 1','Qty 2','Und 2','U$ FOB Tot',
                                'U$ FOB Und 1','U$ FOB Und 2','Pais de Destino','Puerto de destino','Último Puerto Embarque',
                                'Via','Agente Portuario','Agente de Aduana','Descripcion Comercial','Descripcion1','Descripcion2',
                                'Descripcion3','Descripcion4','Descripcion5','Naviera','Agente Carga(Origen)',
                                'Agente Carga(Destino)'])

for index, row in df.iterrows():
    #print(row['Archivo'], row['Fecha'])
    pdxls=pd.read_excel(row['Archivo'])
    pdxls.columns=['Partida Aduanera','Descripcion de la Partida Aduanera','Aduana','DUA','Fecha','Cod. Tributario',
                                'Exportador','Importador','Kg Bruto','Kg Neto','Qty 1','Und 1','Qty 2','Und 2','U$ FOB Tot',
                                'U$ FOB Und 1','U$ FOB Und 2','Pais de Destino','Puerto de destino','Último Puerto Embarque',
                                'Via','Agente Portuario','Agente de Aduana','Descripcion Comercial','Descripcion1','Descripcion2',
                                'Descripcion3','Descripcion4','Descripcion5','Naviera','Agente Carga(Origen)',
                                'Agente Carga(Destino)']
    pdxls = pdxls.iloc[5:]

    pdfinal = pd.concat([pdfinal, pdxls] , ignore_index=True)


In [5]:
from fuzzywuzzy import fuzz,process

def fuzzy_replace(str_a, str_b, orig_str):
    l = len(str_a.split()) # Length to read orig_str chunk by chunk
    splitted = orig_str.split()
    for i in range(len(splitted)-l+1):
        test = " ".join(splitted[i:i+l])
        if fuzz.ratio(str_a, test) > 75: #Using fuzzwuzzy library to test ratio
            before = " ".join(splitted[:i])
            after = " ".join(splitted[i+1:])
            return before+" "+str_b+" "+after #Output will be sandwich of these three strings




c:\users\cjmartinez\.conda\envs\veritrade\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [15]:
nombres=[]
proporcion=[]
dfimpor=pd.read_excel('Importadores.xlsx')

In [7]:
def cambios(mal,bien):
    for i in mal:
        x=process.extractOne(i,bien)
        nombres.append(x[0])
        proporcion.append(x[1])
    return nombres,proporcion

In [8]:
bien=dfimpor['Importadores'].apply(str)
mal=pdfinal['Importador'].apply(str)
corregido,rango=cambios(mal,bien)

In [17]:
pdfinal['Corregido']=pd.Series(corregido)
pdfinal['Rango']=pd.Series(rango)

In [18]:
pdexport=pdfinal[['Importador','Corregido','Rango']]



In [13]:
writer = pd.ExcelWriter('demo.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
pdexport.to_excel(writer, sheet_name='Sheet1', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()